In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from io import StringIO
import pydotplus

# Pipelines
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbPipeline

# Models
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from IPython.display import Image  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


# Preprocessing
from imblearn.under_sampling import NearMiss, InstanceHardnessThreshold
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from imblearn.combine import SMOTEENN 
from imblearn.combine import SMOTETomek

# Metrics, model evaluation
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.metrics import auc, roc_curve, roc_auc_score, precision_recall_curve
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, cross_val_predict
import sklearn.metrics as metrics

In [2]:
plt.rcParams["figure.figsize"] = (10, 10)

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
def model_results_tr_ts(model, X_train, y_train, X_test, y_test, name):
    """
    Function to evaluate one or multiple models and check train and test results with parameters set.
    
    Model fitting is done inside of the function.
    
    The function takes model, split data, and model name and returns the results on test and train sets. 
    Test set: confusion matrix, classification report. 
    Train and test sets: dataframe with accuracy, precision, recall, F1 score, and rocauc 
    score done with cross-validation.
    """
    print(f'**{name}** results', '\n')
    
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test  = model.predict(X_test)
    
    # Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred_test)
    
    # Visualizing confusion matrix
    print(f'Confusion Matrix of test data')
    group_names  = ['TN', 'FP', 'FN', 'TP']
    group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
    labels       = [f'{v1}\n{v2}' for v1, v2 in zip(group_names, group_counts)]
    labels       = np.asarray(labels).reshape(2,2)
    fig, ax      = plt.subplots(figsize=(8,5))
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='BuGn', ax=ax)
    plt.show()
    
    # Classification report
    print(f'Classifictation Report of test data', '\n')
    print(classification_report(y_test, y_pred_test),'\n')
    
    # Main metrics with cross_val_score
    print(f'Main model evaluation metrics with cross validation (train and test data)','\n')
    strat_k_fold   = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracy       = np.mean(cross_val_score(model, X_train, y_train, cv=strat_k_fold, scoring='accuracy'))
    precision      = np.mean(cross_val_score(model, X_train, y_train, cv=strat_k_fold, scoring='precision'))
    recall         = np.mean(cross_val_score(model, X_train, y_train, cv=strat_k_fold, scoring='recall'))
    f1score        = np.mean(cross_val_score(model, X_train, y_train, cv=strat_k_fold, scoring='f1'))
    rocauc         = np.mean(cross_val_score(model, X_train, y_train, cv=strat_k_fold, scoring='roc_auc'))
    
    df_model_train = pd.DataFrame({'data'         : 'train',
                                   'model'        : [name],
                                   'accuracy'     : [accuracy],
                                   'precision'    : [precision],
                                   'recall'       : [recall],
                                   'f1score'      : [f1score],
                                   'rocauc'       : [rocauc]})
   
    accuracy      = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='accuracy'))
    precision     = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='precision'))
    recall        = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='recall'))
    f1score       = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='f1'))
    rocauc        = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='roc_auc'))
    
    df_model_test = pd.DataFrame({'data'         : 'test',
                                  'model'        : [name],
                                  'accuracy'     : [accuracy],
                                  'precision'    : [precision],
                                  'recall'       : [recall],
                                  'f1score'      : [f1score],
                                  'rocauc'       : [rocauc]})
    
    df_model = pd.concat([df_model_train, df_model_test], ignore_index=True)
    
    print(df_model.to_string())

In [6]:

def model_results_test_df(model, X_train, y_train, X_test, y_test, model_name):
    """
    Function to evaluate multiple models on test data with parameters set.
    
    Model fitting is done inside of the function.
    
    The function takes model, split data and the name of the model and returns the results of the test 
    data: dataframe with accuracy, precision, recall, F1 score and rocauc score done with 
    cross-validation.
    """
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    
    # Main metrics with cross_val_score
    strat_k_fold   = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracy       = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='accuracy'))
    precision      = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='precision'))
    recall         = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='recall'))
    f1score        = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='f1'))
    rocauc         = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='roc_auc'))
    
    df_model_test  = pd.DataFrame({'data'         : 'test',
                                   'model'        : [model_name],
                                   'accuracy'     : [accuracy],
                                   'precision'    : [precision],
                                   'recall'       : [recall],
                                   'f1score'      : [f1score],
                                   'rocauc'       : [rocauc]})
    
    return df_model_test

In [7]:
def model_results_test_cm_cr_df(model, X_train, y_train, X_test, y_test, model_name):
    """
    Function to evaluate multiple models on test data with parameters set.
    
    Model fitting is done inside of the function.
    
    The function takes model, test data (X and y) and model name and returns the results on test set:
    confusion matrix, classification report and dataframe with accuracy, precision, recall, F1 score
    and rocauc score done with cross-validation.
    """
    model.fit(X_train, y_train)
    print('\n', f'**{model_name}** results', '\n')
    y_pred_test = model.predict(X_test)
    
    # Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred_test)
    
    # Visualizing confusion matrix
    print(f'Confusion Matrix of test data')
    group_names  = ['TN', 'FP', 'FN', 'TP']
    group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
    labels       = [f'{v1}\n{v2}' for v1, v2 in zip(group_names, group_counts)]
    labels       = np.asarray(labels).reshape(2,2)
    fig, ax      = plt.subplots(figsize=(8,5))
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='BuGn', ax=ax)
    plt.show()
    
    # Classification report
    print(f'Classifictation Report of test data', '\n')
    print(classification_report(y_test, y_pred_test),'\n')
    
    # Main metrics with cross_val_score
    strat_k_fold   = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracy       = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='accuracy'))
    precision      = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='precision'))
    recall         = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='recall'))
    f1score        = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='f1'))
    rocauc         = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='roc_auc'))
    
    df_model_test  = pd.DataFrame({'data'         : 'test',
                                   'model'        : [model_name],
                                   'accuracy'     : [accuracy],
                                   'precision'    : [precision],
                                   'recall'       : [recall],
                                   'f1score'      : [f1score],
                                   'rocauc'       : [rocauc]})
    
    return df_model_test

In [8]:

def model_results_test_cm_df(model, X_train, y_train, X_test, y_test, model_name):
    """
    Function to evaluate multiple models on test data with parameters set.
    
    Model fitting is done inside of the function.
    
    The function takes the model, split data and the model name and returns the results on the 
    test set:
    confusion matrix and the dataframe with accuracy, precision, recall, F1 score
    and rocauc score done with cross-validation.
    """
    model.fit(X_train, y_train)
    print('\n', f'**{model_name}** results', '\n')
    y_pred_test = model.predict(X_test)
    
    # Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred_test)
    
    # Visualizing confusion matrix
    print(f'Confusion Matrix of test data')
    group_names  = ['TN', 'FP', 'FN', 'TP']
    group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
    labels       = [f'{v1}\n{v2}' for v1, v2 in zip(group_names, group_counts)]
    labels       = np.asarray(labels).reshape(2,2)
    fig, ax      = plt.subplots(figsize=(8,5))
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='BuGn', ax=ax)
    plt.show()
    
    # Main metrics with cross_val_score
    strat_k_fold   = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracy       = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='accuracy'))
    precision      = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='precision'))
    recall         = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='recall'))
    f1score        = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='f1'))
    rocauc         = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='roc_auc'))
    
    df_model_test  = pd.DataFrame({'data'         : 'test',
                                   'model'        : [model_name],
                                   'accuracy'     : [accuracy],
                                   'precision'    : [precision],
                                   'recall'       : [recall],
                                   'f1score'      : [f1score],
                                   'rocauc'       : [rocauc]})
    
    return df_model_test

In [9]:

def model_results_hyp_test_cm_cr_df(model, X_test, y_test, model_name):
    """
    Function to evaluate one or multiple models on test set with hyperparameter tuning.
    
    Model fitting is done outside of the function (when doing grid search).
    
    The function takes model, test data (X and y) and model name and returns the results on test set:
    confusion matrix, classification report and dataframe with accuracy, precision, recall, F1 score
    and rocauc score done with cross-validation.
    """
    print('\n', f'**{model_name}** results', '\n')
    y_pred_test = model.predict(X_test)
    
    # Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred_test)
    
    # Visualizing confusion matrix
    print(f'Confusion Matrix of test data')
    group_names  = ['TN', 'FP', 'FN', 'TP']
    group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
    labels       = [f'{v1}\n{v2}' for v1, v2 in zip(group_names, group_counts)]
    labels       = np.asarray(labels).reshape(2,2)
    fig, ax      = plt.subplots(figsize=(8,5))
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='BuGn', ax=ax)
    plt.show()
    
    # Classification report
    print(f'Classifictation Report of test data', '\n')
    print(classification_report(y_test, y_pred_test),'\n')
    
    # Main metrics with cross_val_score
    strat_k_fold   = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracy       = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='accuracy'))
    precision      = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='precision'))
    recall         = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='recall'))
    f1score        = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='f1'))
    rocauc         = np.mean(cross_val_score(model, X_test, y_test, cv=strat_k_fold, scoring='roc_auc'))
    
    df_model_test  = pd.DataFrame({'data'         : 'test',
                                   'model'        : [model_name],
                                   'accuracy'     : [accuracy],
                                   'precision'    : [precision],
                                   'recall'       : [recall],
                                   'f1score'      : [f1score],
                                   'rocauc'       : [rocauc]})
    
    return df_model_test


Introduction
After exploring and preprocessing our data we are ready to start working with Machine Learning algorithms. Before diving into models, let's define what problem we have, what models should be used, and which metrics will be used for evaluation.

We have a binary classification problem, we want to predict if a customer is going to churn after the first month or not (two classes).
We will be using supervised machine learning models. We will train our model with historical data that has an outcome specified.
The best metric in our case is F1 score, which shows the balance between precision and recall. We will also be using the confusion matrix to see how many customers have fallen under correct categories (True Positive and True Negative) and how many were mistaken (False Positive and False Negative). 
In the case of improving precision or recall, we will be focusing on improving recall. High precision relates to a low False Positive rate, and high recall relates to a low False Negative rate. The lower False Negative rate, the better. In our case, False Negative entries indicate that the customers will not churn but actually they will. Thinking that customer is not going to churn, we are losing opportunities because we are not putting appropriate attention to his/hers case. It means that to recover this loss, we would need to invest more, to again acquire a new customer. 

In [10]:
df=pd.read_csv(r"C:\Users\Álvaro\Final-project\Data\df_after_EDA.csv")

df.head()

,eventTime_ay,Date,email,firstName,lastName,Sub_Categoria,campaign,medium,userId,singular_duplicate,test_duplicate,row_numbers,extCode,Comercio,RC,session_count_email,session_count_userId,eventTime_ac,gclid,country,state,town,zipcode,policy_start_date,policy_end_date,floors,goods,surface,inauguration_year,locationRisk,numLocals,shop_postcalcode,refurbishing_year,shop_region,electronic_protections,fire_protections,theft_protections,watchman_protections,Type,eventTime_vg,gross_income,money_invested,furnitureWithEquipment,merchandise,ownership,eventTime_off,monthlypayment1,premiumPrice1,monthlypayment2,premiumPrice2,monthlypayment3,premiumPrice3,provider,eventTime_fd,birthdate,documenttype,offerchosen,billingcycle,eventTime_pm,subcription_id,intent_status,endDate,paymenttype,policynumber,policy_url,policyprice,currentprice,Sales Date,Sub-categoria,Billing Cycle,Policy Number,Policy URL,Current Price,Policy Price,Start Date,Payment Type,Persona,Fecha Anulación,Razon anulación,Aseguradora,Online/offline,Renovado,Precio Renovacion,Tipo de póliza,Unnamed: 86,Match email,Match telefono,Semana,Mes,WP,Dia Lead,quote,sale,Categoria,time_diff_ay_ac,time_diff_ac_vg,time_diff_vg_ff,time_diff_ff_fd,time_diff_fd_pm,city,cityId,regionId,sessions,alimentacion-y-dietetica,arte-y-entretenimiento,automocion,calzado-marroqueria-y-piel,confecciones-y-textiles,construccion-y-reformas,deportes-y-ocio,educacion,electronica-y-maquinaria,fabricacion,hosteleria-y-restauracion,muebles-decoracion-y-menaje-hogar,negocio-inmobiliario,peluquerias-y-centros-estetica,profesionales-economia,profesionales-oficina,rc-tecnicas-y-profesionales,servicios-generales,servicios-sanitarios,transporte-y-almacenamiento,varios,(not set),Andalusia,Andorra la Vella,Aragon,Asturias,Azuay,Balearic Islands,Basque Country,Bavaria,Bogota,Buenos Aires,Canary Islands,Cantabria,Capital District,Cartago Province,Castile and Leon,Castile-La Mancha,Catalonia,Ceuta,Community of Madrid,England,Extremadura,Galicia,Georgia,Hessen,Ile-de-France,La Rioja,Lombardy,Madrid,Melilla,Morelos,Murcia,Navarre,New Hampshire,New York,North Holland,North Rhine-Westphalia,Occitanie,Pays de la Loire,Porto District,Prague,Region of Murcia,Rio Grande,Santa Fe Province,Santiago Metropolitan Region,Scotland,Utrecht,Valencian Community,Vienna,Zulia,GoogleDisplay,SingularC.email.billage,adwords,bing,facebook,hs_automation,hubspot,trustpilot


In [11]:
df.shape

(0, 182)

# Data preprocessing


In [12]:
X = df.drop(columns='sale')
y = df['sale']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4)

print(f'Training data shape X: {X_train.shape}')


print(f'Testing data shape X: {X_test.shape}')

ValueError: With n_samples=0, test_size=0.4 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
ax = sns.countplot(data=df, x='sale', palette={1: "Gold", 0: "Lightblue"})
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()

val_counts = round(df.sale.value_counts(normalize=True)*100, 2)
print(f'The split between two classes (sale and not sale):\n{val_counts}')

# Model selection

In [ ]:

estimator = RandomForestClassifier(criterion='entropy', 
                                   max_depth=2, 
                                   max_features='auto', 
                                   n_estimators=200, 
                                   random_state=42) 

estimator_name = 'RandomForestClassifier'

pipeline = Pipeline([('scaler', StandardScaler()), ('clf', estimator)])
model_results_tr_ts(pipeline, X_train, y_train, X_test, y_test, estimator_name)

In [ ]:
feat_importances.nlargest(25)

In [ ]:
feat_importance.nñargest(25).reset_index().index

In [ ]:
feat_importances = pd.Series(pipeline.steps[1][1].feature_importances_, 
                             index=df.drop(['sale'], axis=1).columns)

feat_importances.nlargest(50).sort_values().plot(kind='barh', color='#1fb299')
plt.show()

In [ ]:
model = RandomForestClassifier(criterion='entropy', 
                               max_depth=2, 
                               max_features='auto', 
                               n_estimators=200, 
                               random_state=42) 
# Train
model.fit(X_train, y_train)

# Extract single tree
estimator = model.estimators_[2]

# Export as dot file
dot_data = StringIO()
export_graphviz(estimator, out_file=dot_data, 
                feature_names = df.drop(['sale'], axis=1).columns,
                class_names = ['not_churned', 'churned'],
                rounded=True, proportion=False, precision=2, filled=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png')

In [ ]:

# The lists of the important features

imp_features_to_filter_short = ['newsletter', 'master_channel_f', 'registration_payment', 
                                'with_promotion', 'sex', 'master_channel_a','master_channel_e', 
                                'master_channel_de', 'age', 'income_per_region', 'master_channel_s', 
                                'email_domain_ce', 'reg_year', 'churned_1_month']

imp_features_to_filter_long = ['newsletter', 'master_channel_f', 'registration_payment', 
                               'with_promotion', 'sex', 'master_channel_a','master_channel_e', 
                               'master_channel_de', 'age', 'income_per_region', 'master_channel_s', 
                               'email_domain_ce', 'email_domain_oe', 'is_lead', 'population_per_region',
                               'days_from_first_click', 'email_domain_ge', 'email_domain_ae', 
                               'box_satisfaction', 'reg_month_b', 'master_channel_ps', 'reg_month_c',
                               'email_domain_hc', 'reg_month_k', 'reg_year', 'churned_1_month']

In [ ]:
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

In [ ]:
models = {'RandomForestClassifier': RandomForestClassifier(random_state=42),
          'ExtraTreesClassifier': ExtraTreesClassifier(random_state=42), 
          'LogisticRegression': LogisticRegression(random_state=42), 
          'SVC': SVC(random_state=42),
          'KNeighborsClassifier': KNeighborsClassifier(),
          'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
          'LinearSVC': LinearSVC(random_state=42),
         }

models_df_test = pd.concat([model_results_test_df(model, X_train, y_train, X_test, y_test, name) for (name, model) in models.items()])
models_df_test.reset_index(inplace=True, drop=True)
models_df_test

# SMOTE 

Synthetic Minority Oversampling Technique (SMOTE). It works by selecting examples that are close in the feature space, drawing a line between the examples in the feature space and drawing a new sample at a point along that line.


In [ ]:
oversamp_undersamp = SMOTETomek(sampling_strategy=0.6) #default sampling strategy
X_train_b, y_train_b = oversamp_undersamp.fit_sample(X_train, y_train)
X_train_b.shape, y_train_b.shape
y_train_b.value_counts()

In [ ]:

estimator = RandomForestClassifier(criterion='entropy', 
                                   max_depth=2, 
                                   max_features='auto', 
                                   n_estimators=200, 
                                   random_state=42) 

estimator_name = 'RandomForestClassifier'

pipeline = Pipeline([('scaler', StandardScaler()), ('clf', estimator)])
model_results_tr_ts(pipeline, X_train_b, y_train_b, X_test, y_test, estimator_name)

In [ ]:
models = {'RandomForestClassifier': RandomForestClassifier(random_state=42),
          'ExtraTreesClassifier': ExtraTreesClassifier(random_state=42), 
          'LogisticRegression': LogisticRegression(random_state=42), 
          'SVC': SVC(random_state=42),
          'KNeighborsClassifier': KNeighborsClassifier(),
          'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
          'LinearSVC': LinearSVC(random_state=42),
         }

models_df_test = pd.concat([model_results_test_df(model, X_train_b, y_train_b, X_test, y_test, name) for (name, model) in models.items()])
models_df_test.reset_index(inplace=True, drop=True)
models_df_test

In [ ]:
models = {'RandomForestClassifier': RandomForestClassifier(random_state=42),
          'ExtraTreesClassifier': ExtraTreesClassifier(random_state=42),
          'LogisticRegression': LogisticRegression(random_state=42), 
          'SVC': SVC(random_state=42),
          'KNeighborsClassifier': KNeighborsClassifier(),
          'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
          'LinearSVC': LinearSVC(random_state=42),
         }

# Defining values for the parameters
random_grid = {'n_estimators': [100, 200, 300],
              'max_features': ['auto', 'sqrt', 'log2'],
              'max_depth' : [2, 5, 10],
              'criterion' :['gini', 'entropy']}

xgb_grid   = {'learning_rate': [0.01],
              'n_estimators': [300, 700],
              'max_depth':[7, 8],
              'reg_alpha':[0.3, 0.4]}

extra_grid = {'n_estimators': [20, 50, 100],
             'max_features': ['auto', 'sqrt', 'log2'],
             'min_samples_split': [2, 4, 6]}

lgbm_grid  = {'num_leaves': [10, 30, 50], 
             'min_child_samples': [20, 40, 60], 
             'min_child_weight': [1e-5, 1e-3, 1e-2],
             'reg_alpha': [1e-1, 5],
             'reg_lambda': [1e-1, 5]}

c_values   = list(10.0**np.arange(-3, 3))
log_grid   = [{'C': c_values,
              'penalty': ['l2'],
              'solver' : ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'],
              'multi_class': ['ovr', 'warn']},
              {'C': c_values,
              'penalty': ['l1'],
              'solver' : ['liblinear', 'saga'],
              'multi_class': ['ovr', 'warn']}]

svc_grid   = {'C': [0.1, 1],
              'gamma': [1, 0.1],
              'kernel': ['rbf']}

knn_grid   = {'n_neighbors': [5,7,9,11,13,15,17,19],
              'weights': ['uniform', 'distance'],
              'metric' : ['euclidean', 'manhattan']}

depth      = list(np.arange(2, 10))
dtree_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
              'max_depth' : depth}

c_values   = list(10.0**np.arange(-3, 3))
lsvc_grid  = [{'C': c_values,
              'multi_class': ['ovr', 'crammer_singer'],
              'penalty': ['l1'],
              'loss': ['squared_hinge'],
              'dual': [0]},
              {'C': c_values,
              'multi_class': ['ovr', 'crammer_singer'],
              'penalty': ['l2'],
              'loss': ['squared_hinge'],
              'dual': [1]}]


grids = [random_grid, extra_grid, log_grid, svc_grid, knn_grid, dtree_grid,lsvc_grid]

# Defining cv
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

appended_dfs = []
for (name, model), param_grid in zip (models.items(), grids):
    # Runing grid search
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=strat_k_fold, n_jobs=-1, 
                        scoring='f1')
    # Fitting the grid and use it to find the best estimator on train data
    grid.fit(X_train_b, y_train_b) 
    # Creating the variable for best etimator 
    best_estimator = grid.best_estimator_
    # Conducting model metrics on test data
    models_df_test = model_results_hyp_test_cm_cr_df(best_estimator, X_test, y_test, name)
    # Printing best parameters
    print(f'Best parameters: {grid.best_params_}')
    appended_dfs.append(models_df_test)

model_results_hyp = pd.concat(appended_dfs, ignore_index = True)
model_results_hyp
